In [0]:

# MAGIC %md
# MAGIC # 07 - Model Management Dashboard
# MAGIC 
# MAGIC **Centralized dashboard for managing models, versions, and experiments**
# MAGIC 
# MAGIC ## Objectives:
# MAGIC - View all registered models and versions
# MAGIC - Compare model performance across versions
# MAGIC - Manage model stages (Staging, Production, Archived)
# MAGIC - View experiment runs and metrics
# MAGIC - Clean up old models and artifacts


In [0]:
%restart_python

In [0]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# MLflow
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Imports complete")

In [0]:
# MAGIC %md
# MAGIC ## 2. Project Setup

In [0]:
print("="*60)
print("MODEL MANAGEMENT DASHBOARD")
print("="*60)

# Define project root
project_root = "/Workspace/COMM - Commercial Analytics (CMAN)/MMM Quattro 2025/Satish/MLFLOW_sample"

# Add to path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"\n📂 Project root: {project_root}")

# Import custom modules
from src.utils import ConfigLoader, safe_display

print(f"✅ Custom modules imported")
print("="*60)

In [0]:
# MAGIC %md
# MAGIC ## 3. Load Configuration & Setup MLflow

In [0]:
print("="*60)
print("LOADING CONFIGURATION")
print("="*60)

config_path = f'{project_root}/config/config.yaml'
config = ConfigLoader.load_config(config_path)

experiment_name = config['mlflow']['experiment_name']
model_registry_name = config['mlflow']['model_registry_name']

mlflow.set_experiment(experiment_name)
client = MlflowClient()

print(f"\n✅ Configuration loaded")
print(f"  • Experiment: {experiment_name}")
print(f"  • Model Registry: {model_registry_name}")
print("="*60)

In [0]:
# MAGIC %md
# MAGIC ## 4. View All Registered Models

In [0]:
print("="*60)
print("REGISTERED MODELS OVERVIEW")
print("="*60)

# Get all registered models
registered_models = client.search_registered_models()

if registered_models:
    models_data = []
    
    for rm in registered_models:
        # Get latest versions
        versions = client.search_model_versions(f"name='{rm.name}'")
        
        models_data.append({
            'Model Name': rm.name,
            'Total Versions': len(versions),
            'Latest Version': versions[0].version if versions else 'N/A',
            'Creation Time': datetime.fromtimestamp(rm.creation_timestamp / 1000).strftime('%Y-%m-%d %H:%M'),
            'Last Updated': datetime.fromtimestamp(rm.last_updated_timestamp / 1000).strftime('%Y-%m-%d %H:%M'),
            'Description': rm.description[:50] + '...' if rm.description and len(rm.description) > 50 else rm.description or 'N/A'
        })
    
    models_df = pd.DataFrame(models_data)
    
    print(f"\n📊 Total Registered Models: {len(registered_models)}")
    safe_display(models_df)
else:
    print(f"\n⚠️ No registered models found")

print("="*60)

In [0]:
# MAGIC %md
# MAGIC ## 5. View Model Versions by Stage

In [0]:
print("="*60)
print("MODEL VERSIONS BY STAGE")
print("="*60)

try:
    # Get all versions of the main model
    all_versions = client.search_model_versions(f"name='{model_registry_name}'")
    
    if all_versions:
        versions_data = []
        
        for version in all_versions:
            # Get run details
            try:
                run = client.get_run(version.run_id)
                metrics = run.data.metrics
                params = run.data.params
                
                versions_data.append({
                    'Version': version.version,
                    'Stage': version.current_stage,
                    'Status': version.status,
                    'Created': datetime.fromtimestamp(version.creation_timestamp / 1000).strftime('%Y-%m-%d %H:%M'),
                    'RMSE': f"${metrics.get('test_rmse', 0):,.2f}" if 'test_rmse' in metrics else 'N/A',
                    'MAE': f"${metrics.get('test_mae', 0):,.2f}" if 'test_mae' in metrics else 'N/A',
                    'R²': f"{metrics.get('test_r2', 0):.4f}" if 'test_r2' in metrics else 'N/A',
                    'Model Type': params.get('model_type', 'N/A'),
                    'Run ID': version.run_id[:8] + '...'
                })
            except:
                versions_data.append({
                    'Version': version.version,
                    'Stage': version.current_stage,
                    'Status': version.status,
                    'Created': datetime.fromtimestamp(version.creation_timestamp / 1000).strftime('%Y-%m-%d %H:%M'),
                    'RMSE': 'N/A',
                    'MAE': 'N/A',
                    'R²': 'N/A',
                    'Model Type': 'N/A',
                    'Run ID': version.run_id[:8] + '...'
                })
        
        versions_df = pd.DataFrame(versions_data)
        
        print(f"\n📊 Model: {model_registry_name}")
        print(f"📊 Total Versions: {len(all_versions)}")
        
        # Group by stage
        for stage in ['Production', 'Staging', 'Archived', 'None']:
            stage_versions = versions_df[versions_df['Stage'] == stage]
            if len(stage_versions) > 0:
                print(f"\n{'='*60}")
                print(f"📌 {stage} ({len(stage_versions)} version{'s' if len(stage_versions) > 1 else ''})")
                print(f"{'='*60}")
                safe_display(stage_versions)
        
    else:
        print(f"\n⚠️ No versions found for model: {model_registry_name}")

except Exception as e:
    print(f"\n❌ Error retrieving model versions: {e}")

print("="*60)

In [0]:
# MAGIC %md
# MAGIC ## 6. Compare Model Performance Across Versions

In [0]:
print("="*60)
print("MODEL PERFORMANCE COMPARISON")
print("="*60)

try:
    all_versions = client.search_model_versions(f"name='{model_registry_name}'")
    
    if all_versions and len(all_versions) > 1:
        comparison_data = []
        
        for version in all_versions:
            try:
                run = client.get_run(version.run_id)
                metrics = run.data.metrics
                
                comparison_data.append({
                    'Version': int(version.version),
                    'Stage': version.current_stage,
                    'RMSE': metrics.get('test_rmse', np.nan),
                    'MAE': metrics.get('test_mae', np.nan),
                    'R2': metrics.get('test_r2', np.nan),
                    'Created': datetime.fromtimestamp(version.creation_timestamp / 1000)
                })
            except:
                pass
        
        if comparison_data:
            comparison_df = pd.DataFrame(comparison_data).sort_values('Version')
            
            # Plot performance trends
            fig, axes = plt.subplots(1, 3, figsize=(16, 5))
            
            # RMSE trend
            axes[0].plot(comparison_df['Version'], comparison_df['RMSE'], marker='o', linewidth=2, markersize=8)
            for idx, row in comparison_df.iterrows():
                if row['Stage'] == 'Production':
                    axes[0].scatter(row['Version'], row['RMSE'], color='red', s=200, zorder=5, marker='*', label='Production')
            axes[0].set_xlabel('Version', fontsize=11, fontweight='bold')
            axes[0].set_ylabel('RMSE ($)', fontsize=11, fontweight='bold')
            axes[0].set_title('RMSE Across Versions', fontsize=12, fontweight='bold')
            axes[0].grid(True, alpha=0.3)
            axes[0].legend()
            
            # MAE trend
            axes[1].plot(comparison_df['Version'], comparison_df['MAE'], marker='o', linewidth=2, markersize=8, color='orange')
            for idx, row in comparison_df.iterrows():
                if row['Stage'] == 'Production':
                    axes[1].scatter(row['Version'], row['MAE'], color='red', s=200, zorder=5, marker='*', label='Production')
            axes[1].set_xlabel('Version', fontsize=11, fontweight='bold')
            axes[1].set_ylabel('MAE ($)', fontsize=11, fontweight='bold')
            axes[1].set_title('MAE Across Versions', fontsize=12, fontweight='bold')
            axes[1].grid(True, alpha=0.3)
            axes[1].legend()
            
            # R² trend
            axes[2].plot(comparison_df['Version'], comparison_df['R2'], marker='o', linewidth=2, markersize=8, color='green')
            for idx, row in comparison_df.iterrows():
                if row['Stage'] == 'Production':
                    axes[2].scatter(row['Version'], row['R2'], color='red', s=200, zorder=5, marker='*', label='Production')
            axes[2].set_xlabel('Version', fontsize=11, fontweight='bold')
            axes[2].set_ylabel('R² Score', fontsize=11, fontweight='bold')
            axes[2].set_title('R² Across Versions', fontsize=12, fontweight='bold')
            axes[2].grid(True, alpha=0.3)
            axes[2].legend()
            
            plt.tight_layout()
            plt.show()
            
            # Show comparison table
            print(f"\n📊 Performance Comparison Table:")
            comparison_display = comparison_df.copy()
            comparison_display['RMSE'] = comparison_display['RMSE'].apply(lambda x: f"${x:,.2f}" if not pd.isna(x) else 'N/A')
            comparison_display['MAE'] = comparison_display['MAE'].apply(lambda x: f"${x:,.2f}" if not pd.isna(x) else 'N/A')
            comparison_display['R2'] = comparison_display['R2'].apply(lambda x: f"{x:.4f}" if not pd.isna(x) else 'N/A')
            comparison_display['Created'] = comparison_display['Created'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M'))
            safe_display(comparison_display)
            
            print(f"\n✅ Performance comparison complete")
        else:
            print(f"\n⚠️ No metrics available for comparison")
    else:
        print(f"\n⚠️ Need at least 2 versions for comparison")

except Exception as e:
    print(f"\n❌ Error comparing models: {e}")
    import traceback
    traceback.print_exc()

print("="*60)

In [0]:
# MAGIC %md
# MAGIC ## 7. View Recent Experiment Runs

In [0]:
print("="*60)
print("RECENT EXPERIMENT RUNS")
print("="*60)

try:
    experiment = mlflow.get_experiment_by_name(experiment_name)
    
    # Get recent runs
    runs = client.search_runs(
        experiment_ids=[experiment.experiment_id],
        order_by=["start_time DESC"],
        max_results=10
    )
    
    if runs:
        runs_data = []
        
        for run in runs:
            runs_data.append({
                'Run Name': run.data.tags.get('mlflow.runName', 'N/A'),
                'Status': run.info.status,
                'Start Time': datetime.fromtimestamp(run.info.start_time / 1000).strftime('%Y-%m-%d %H:%M'),
                'Duration (s)': f"{(run.info.end_time - run.info.start_time) / 1000:.1f}" if run.info.end_time else 'Running',
                'RMSE': f"${run.data.metrics.get('test_rmse', 0):,.2f}" if 'test_rmse' in run.data.metrics else 'N/A',
                'R²': f"{run.data.metrics.get('test_r2', 0):.4f}" if 'test_r2' in run.data.metrics else 'N/A',
                'Model': run.data.params.get('model_type', 'N/A'),
                'Run ID': run.info.run_id[:8] + '...'
            })
        
        runs_df = pd.DataFrame(runs_data)
        
        print(f"\n📊 Last 10 Runs:")
        safe_display(runs_df)
        
        print(f"\n✅ Total runs in experiment: {len(runs)}")
    else:
        print(f"\n⚠️ No runs found in experiment")

except Exception as e:
    print(f"\n❌ Error retrieving runs: {e}")

print("="*60)

In [0]:
# MAGIC %md
# MAGIC ## 8. Model Stage Management

In [0]:
print("="*60)
print("MODEL STAGE MANAGEMENT")
print("="*60)

print(f"\n📋 Available Actions:")
print(f"  1. Promote model to Staging")
print(f"  2. Promote model to Production")
print(f"  3. Archive model version")
print(f"  4. Delete model version")

print(f"\n💡 Example Usage:")
print(f"""
# Promote version 2 to Production
client.transition_model_version_stage(
    name='{model_registry_name}',
    version='2',
    stage='Production',
    archive_existing_versions=True
)

# Archive version 1
client.transition_model_version_stage(
    name='{model_registry_name}',
    version='1',
    stage='Archived'
)

# Delete version 3
client.delete_model_version(
    name='{model_registry_name}',
    version='3'
)
""")

print(f"\n⚠️ Uncomment and modify the code below to perform actions:")

# EXAMPLE: Uncomment to use
# version_to_promote = '2'
# client.transition_model_version_stage(
#     name=model_registry_name,
#     version=version_to_promote,
#     stage='Production',
#     archive_existing_versions=True
# )
# print(f"✅ Version {version_to_promote} promoted to Production")

print("="*60)

In [0]:
# MAGIC %md
# MAGIC ## 9. Cleanup Old Artifacts

In [0]:
print("="*60)
print("CLEANUP OLD ARTIFACTS")
print("="*60)

print(f"\n📋 Cleanup Options:")
print(f"  1. Delete archived model versions")
print(f"  2. Delete old experiment runs")
print(f"  3. Clean up artifacts older than X days")

print(f"\n💡 Example: Delete all archived versions")
print(f"""
# Get all archived versions
archived_versions = [v for v in client.search_model_versions(f"name='{model_registry_name}'") 
                     if v.current_stage == 'Archived']

# Delete each archived version
for version in archived_versions:
    client.delete_model_version(
        name='{model_registry_name}',
        version=version.version
    )
    print(f"Deleted version {{version.version}}")
""")

print(f"\n⚠️ Use with caution - deletions are permanent!")

# EXAMPLE: Uncomment to delete archived versions
# archived_versions = [v for v in client.search_model_versions(f"name='{model_registry_name}'") 
#                      if v.current_stage == 'Archived']
# 
# if archived_versions:
#     print(f"\n🗑️ Found {len(archived_versions)} archived versions")
#     for version in archived_versions:
#         # Uncomment to actually delete
#         # client.delete_model_version(name=model_registry_name, version=version.version)
#         print(f"  • Version {version.version} (would be deleted)")
# else:
#     print(f"\n✅ No archived versions to clean up")

print("="*60)

In [0]:
# MAGIC %md
# MAGIC ## 10. Dashboard Summary

In [0]:
print("="*60)
print("DASHBOARD SUMMARY")
print("="*60)

try:
    # Get summary statistics
    all_models = client.search_registered_models()
    all_versions = client.search_model_versions(f"name='{model_registry_name}'")
    experiment = mlflow.get_experiment_by_name(experiment_name)
    all_runs = client.search_runs(experiment_ids=[experiment.experiment_id])
    
    # Count by stage
    stage_counts = {}
    for version in all_versions:
        stage = version.current_stage
        stage_counts[stage] = stage_counts.get(stage, 0) + 1
    
    print(f"\n📊 MLflow Registry Overview:")
    print(f"  • Total Registered Models: {len(all_models)}")
    print(f"  • Total Model Versions: {len(all_versions)}")
    print(f"  • Production Versions: {stage_counts.get('Production', 0)}")
    print(f"  • Staging Versions: {stage_counts.get('Staging', 0)}")
    print(f"  • Archived Versions: {stage_counts.get('Archived', 0)}")
    
    print(f"\n📊 Experiment Overview:")
    print(f"  • Experiment Name: {experiment_name}")
    print(f"  • Total Runs: {len(all_runs)}")
    
    # Get production model info
    production_versions = [v for v in all_versions if v.current_stage == 'Production']
    if production_versions:
        prod_version = production_versions[0]
        prod_run = client.get_run(prod_version.run_id)
        
        print(f"\n🚀 Current Production Model:")
        print(f"  • Version: {prod_version.version}")
        print(f"  • RMSE: ${prod_run.data.metrics.get('test_rmse', 0):,.2f}")
        print(f"  • MAE: ${prod_run.data.metrics.get('test_mae', 0):,.2f}")
        print(f"  • R²: {prod_run.data.metrics.get('test_r2', 0):.4f}")
        print(f"  • Deployed: {datetime.fromtimestamp(prod_version.creation_timestamp / 1000).strftime('%Y-%m-%d %H:%M')}")
    
    print(f"\n✅ Dashboard loaded successfully")

except Exception as e:
    print(f"\n❌ Error loading summary: {e}")

print("="*60)



Typical Workflow
Here's how you'd use the dashboard in practice:

Scenario 1: New Model is Better - Promote to Production
Run the dashboard to see all versions
Check Section 6 - Compare performance
See that version 3 has better metrics (lower RMSE, higher R²)
Go to Section 8 - Uncomment promotion code
Promote version 3 to Production
Re-run Section 5 to verify the change
Scenario 2: Clean Up Old Models
Run the dashboard
Check Section 5 - See you have 5 archived versions
Go to Section 9 - Uncomment cleanup code
Run the cell to delete archived versions
Re-run Section 4 to verify cleanup
Scenario 3: Compare Model Performance Over Time
Run the dashboard
Look at Section 6 charts
Analyze trends:
Is performance improving with each version?
Which version is the best?
Should you promote a staging model?

# 1. Promote to Production
client.transition_model_version_stage(
    name='house_price_predictor',
    version='3',
    stage='Production',
    archive_existing_versions=True
)

# 2. Move to Staging
client.transition_model_version_stage(
    name='house_price_predictor',
    version='4',
    stage='Staging'
)

# 3. Archive a version
client.transition_model_version_stage(
    name='house_price_predictor',
    version='1',
    stage='Archived'
)

# 4. Delete a version (permanent!)
client.delete_model_version(
    name='house_price_predictor',
    version='1'
)

# 5. Get all versions
versions = client.search_model_versions(f"name='house_price_predictor'")
for v in versions:
    print(f"Version {v.version}: {v.current_stage}")